In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("JupyterLoadFlightsToPostgres")
    .master("spark://spark-master:7077")
    .config("spark.jars", "/opt/spark/jars/postgresql.jar")
    .getOrCreate()
)


df = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .option("nullValue", "NA")        # "NA" → null
    .csv("/data/flights-larger.csv")
)

df.show(5)
df.printSchema()
df.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres-dwh:5432/dwhdb") \
    .option("dbtable", "flights") \
    .option("user", "dwh") \
    .option("password", "dwh123") \
    .option("driver", "org.postgresql.Driver") \
    .mode("append") \
    .save()

spark.stop()

# kiem tra db:
# docker compose exec postgres-dwh psql -U dwh -d dwhdb
# SELECT COUNT(*) FROM flights;
# SELECT * FROM flights LIMIT 10;

[11:27:55] [WARN] NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
                                                                                

+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 10| 10|  1|     OO|  5836|ORD| 157|  8.18|      51|   27|
|  1|  4|  1|     OO|  5866|ORD| 466|  15.5|     102| NULL|
| 11| 22|  1|     OO|  6016|ORD| 738|  7.17|     127|  -19|
|  2| 14|  5|     B6|   199|JFK|2248| 21.17|     365|   60|
|  5| 25|  3|     WN|  1675|SJC| 386| 12.92|      85|   22|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows

root
 |-- mon: integer (nullable = true)
 |-- dom: integer (nullable = true)
 |-- dow: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- org: string (nullable = true)
 |-- mile: integer (nullable = true)
 |-- depart: double (nullable = true)
 |-- duration: integer (nullable = true)
 |-- delay: integer (nullable = true)

